In [5]:
print('HELLO Marius')



HELLO Marius


In [ ]:
%pip install langchain langchain-openai langchain_chroma langchain_community langchainhub


In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

secret = os.getenv("SECRET")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, base_url=endpoint,api_key=secret)


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths= str("https://lilianweng.github.io/posts/2017-06-21-overview/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(
    base_url='https://models.inference.ai.azure.com',
    api_key=secret, 
    model="text-embedding-3-small",
))
# Create a retriever from the vector
